<a href="https://colab.research.google.com/github/jeongname/jeongname/blob/main/220405_hj_model1%2C2%2C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 325 kB 5.0 MB/s 
     |████████████████████████████████| 3.8 MB 71.7 MB/s 
     |████████████████████████████████| 212 kB 83.1 MB/s 
     |████████████████████████████████| 1.1 MB 72.2 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 136 kB 79.6 MB/s 
     |████████████████████████████████| 127 kB 73.2 MB/s 
     |████████████████████████████████| 144 kB 76.6 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 271 kB 84.1 MB/s 
     |████████████████████████████████| 895 kB 76.8 MB/s 
     |████████████████████████████████| 596 kB 84.1 MB/s 
     |████████████████████████████████| 6.5 MB 78.4 MB/s 
     |████████████████████████████████| 1.2 MB 73.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

- TPU 사용을 위한 accelerate

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher
from pathlib import Path
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [6]:
df = df.fillna('')

In [7]:
df = df.sample(n=10000)
df = df.reset_index(drop=True)

In [8]:
df_train = df[:9000]
df_test_1 = df[9000:]
df_test = df_test_1

In [9]:
df_test['text'] = df_test['text_obj'].astype(str) + " " + df_test['text_mthd'].astype(str) + " " + df_test['text_deal'].astype(str)
df_test = df_test[['text']]
df_test = df_test.reset_index(drop=True)

### 1. digit_2 예측하기

In [10]:
df = df_train

In [11]:
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0737094,G,46,467,사업장에서,산업사용자에게,비철판매,사업장에서 산업사용자에게 비철판매,G 46
1,id_0610145,G,47,475,사업장에서,일반인대상,액자,사업장에서 일반인대상 액자,G 47
2,id_0479687,I,56,562,커피숍에서,접객시설을갖ㅊ추고,"커피,음료","커피숍에서 접객시설을갖ㅊ추고 커피,음료",I 56
3,id_0236310,S,96,961,미용실,미용실,미용실,미용실 미용실 미용실,S 96
4,id_0213452,S,95,953,사업체에서,일반인을 대상으로,에어콘 수리,사업체에서 일반인을 대상으로 에어콘 수리,S 95
...,...,...,...,...,...,...,...,...,...
8995,id_0160917,S,94,949,교회,기독교계통 종교활동,종교서비스,교회 기독교계통 종교활동 종교서비스,S 94
8996,id_0391660,I,56,561,양식 전문,,,양식 전문,I 56
8997,id_0357429,F,41,412,건설현장에서,,상하수도 공사,건설현장에서 상하수도 공사,F 41
8998,id_0466493,I,56,561,음식점에서,접객시설을 갖추고,된장찌개,음식점에서 접객시설을 갖추고 된장찌개,I 56


In [12]:
train_digit2 = df[['text', 'target']]
train_digit2

,text,target
0,사업장에서 산업사용자에게 비철판매,G 46
1,사업장에서 일반인대상 액자,G 47
2,"커피숍에서 접객시설을갖ㅊ추고 커피,음료",I 56
3,미용실 미용실 미용실,S 96
4,사업체에서 일반인을 대상으로 에어콘 수리,S 95
...,...,...
8995,교회 기독교계통 종교활동 종교서비스,S 94
8996,양식 전문,I 56
8997,건설현장에서 상하수도 공사,F 41
8998,음식점에서 접객시설을 갖추고 된장찌개,I 56


### digit_3 예측하기

In [13]:
df = df_train

In [14]:
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str)+ " " + df['digit_3'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0737094,G,46,467,사업장에서,산업사용자에게,비철판매,사업장에서 산업사용자에게 비철판매,G 46 467
1,id_0610145,G,47,475,사업장에서,일반인대상,액자,사업장에서 일반인대상 액자,G 47 475
2,id_0479687,I,56,562,커피숍에서,접객시설을갖ㅊ추고,"커피,음료","커피숍에서 접객시설을갖ㅊ추고 커피,음료",I 56 562
3,id_0236310,S,96,961,미용실,미용실,미용실,미용실 미용실 미용실,S 96 961
4,id_0213452,S,95,953,사업체에서,일반인을 대상으로,에어콘 수리,사업체에서 일반인을 대상으로 에어콘 수리,S 95 953
...,...,...,...,...,...,...,...,...,...
8995,id_0160917,S,94,949,교회,기독교계통 종교활동,종교서비스,교회 기독교계통 종교활동 종교서비스,S 94 949
8996,id_0391660,I,56,561,양식 전문,,,양식 전문,I 56 561
8997,id_0357429,F,41,412,건설현장에서,,상하수도 공사,건설현장에서 상하수도 공사,F 41 412
8998,id_0466493,I,56,561,음식점에서,접객시설을 갖추고,된장찌개,음식점에서 접객시설을 갖추고 된장찌개,I 56 561


In [15]:
train_digit3 = df[['text', 'target']]
train_digit3

,text,target
0,사업장에서 산업사용자에게 비철판매,G 46 467
1,사업장에서 일반인대상 액자,G 47 475
2,"커피숍에서 접객시설을갖ㅊ추고 커피,음료",I 56 562
3,미용실 미용실 미용실,S 96 961
4,사업체에서 일반인을 대상으로 에어콘 수리,S 95 953
...,...,...
8995,교회 기독교계통 종교활동 종교서비스,S 94 949
8996,양식 전문,I 56 561
8997,건설현장에서 상하수도 공사,F 41 412
8998,음식점에서 접객시설을 갖추고 된장찌개,I 56 561


# 토크나이저, 함수

In [16]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [17]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

### digit_2 예측하기

In [18]:
my_dict2 = {}
for i in range(len(train_digit2['target'].unique())):
  my_dict2[train_digit2['target'].unique()[i]] = i

def target_to_num_digit2(target:str):
  return my_dict2[target]

train_digit2['target'] = train_digit2['target'].map(target_to_num_digit2)
train_digit2

,text,target
0,사업장에서 산업사용자에게 비철판매,0
1,사업장에서 일반인대상 액자,1
2,"커피숍에서 접객시설을갖ㅊ추고 커피,음료",2
3,미용실 미용실 미용실,3
4,사업체에서 일반인을 대상으로 에어콘 수리,4
...,...,...
8995,교회 기독교계통 종교활동 종교서비스,17
8996,양식 전문,2
8997,건설현장에서 상하수도 공사,29
8998,음식점에서 접객시설을 갖추고 된장찌개,2


In [19]:
my_dict2[''] = len(my_dict2)

In [20]:
train_digit2[train_digit2['text'].isnull()]

,text,target


### digit_3 예측하기

In [21]:
my_dict3 = {}
for i in range(len(train_digit3['target'].unique())):
  my_dict3[train_digit3['target'].unique()[i]] = i

def target_to_num_digit3(target:str):
  return my_dict3[target]

train_digit3['target'] = train_digit3['target'].map(target_to_num_digit3)
train_digit3

,text,target
0,사업장에서 산업사용자에게 비철판매,0
1,사업장에서 일반인대상 액자,1
2,"커피숍에서 접객시설을갖ㅊ추고 커피,음료",2
3,미용실 미용실 미용실,3
4,사업체에서 일반인을 대상으로 에어콘 수리,4
...,...,...
8995,교회 기독교계통 종교활동 종교서비스,27
8996,양식 전문,15
8997,건설현장에서 상하수도 공사,47
8998,음식점에서 접객시설을 갖추고 된장찌개,15


In [22]:
my_dict3[''] = len(my_dict3)

In [23]:
train_digit3[train_digit3['text'].isnull()]

,text,target


In [24]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [56]:
train_digit3['target'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

### digit_2 예측하기

In [34]:
train_digit2['target'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67])

In [25]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train_digit2['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220404_inference_digit2/model' + str(fold), save_function=accelerator.save)

In [26]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train_digit2['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_digit2, train_digit2['target'])):
  my_file = Path("/content/drive/MyDrive/220404_inference_digit2/model"+str(fold))
  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train_digit2['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

...

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

epoch 1 train acc 0.16163793103448276
epoch 1 train acc 0.16810344827586207
epoch 1 train acc 0.15625
epoch 1 train acc 0.1788793103448276
epoch 1 train acc 0.17780172413793102
epoch 1 train acc 0.18211206896551724
epoch 1 train acc 0.14439655172413793
epoch 1 train acc 0.15625
epoch 1 test acc 0.29296875
epoch 1 test acc 0.33984375
epoch 1 test acc 0.33203125
epoch 1 test acc 0.36328125
epoch 1 test acc 0.3671875
epoch 1 test acc 0.3515625
epoch 1 test acc 0.3671875
epoch 1 test acc 0.29296875
epoch 2 train acc 0.31788793103448276
epoch 2 train acc 0.35560344827586204
epoch 2 train acc 0.31573275862068967
epoch 2 train acc 0.3426724137931034
epoch 2 train acc 0.32435344827586204
epoch 2 train acc 0.31573275862068967
epoch 2 train acc 0.3459051724137931
epoch 2 train acc 0.3329741379310345
epoch 2 test acc 0.33984375
epoch 2 test acc 0.36328125
epoch 2 test acc 0.3984375
epoch 2 test acc 0.44140625
epoch 2 test acc 0.375
epoch 2 test acc 0.3515625
epoch 2 test acc 0.38671875
epoch 2 te

### digit_3 예측하기

In [27]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train_digit3['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220404_inference_digit3/model' + str(fold), save_function=accelerator.save)

In [28]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train_digit3['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_digit3, train_digit3['target'])):
  my_file = Path("/content/drive/MyDrive/220404_inference_digit3/model"+str(fold))
  if my_file.is_dir():
    continue

  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label3 = train_digit3['target']

  train_dataset3 = BERTDataset(tokenized_train, train_label3)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

...

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

epoch 1 train acc 0.1584051724137931
epoch 1 train acc 0.16594827586206898
epoch 1 train acc 0.1627155172413793
epoch 1 train acc 0.17025862068965517
epoch 1 train acc 0.16918103448275862
epoch 1 train acc 0.17349137931034483
epoch 1 train acc 0.1713362068965517
epoch 1 train acc 0.16702586206896552
epoch 1 test acc 0.31640625
epoch 1 test acc 0.30859375
epoch 1 test acc 0.359375
epoch 1 test acc 0.25
epoch 1 test acc 0.32421875
epoch 1 test acc 0.28125
epoch 1 test acc 0.3359375
epoch 1 test acc 0.3125
epoch 2 train acc 0.2855603448275862
epoch 2 train acc 0.2823275862068966
epoch 2 train acc 0.2640086206896552
epoch 2 train acc 0.30495689655172414
epoch 2 train acc 0.2834051724137931
epoch 2 train acc 0.27262931034482757
epoch 2 train acc 0.2920258620689655
epoch 2 train acc 0.2769396551724138
epoch 2 test acc 0.34765625
epoch 2 test acc 0.3671875
epoch 2 test acc 0.3046875
epoch 2 test acc 0.34375
epoch 2 test acc 0.39453125
epoch 2 test acc 0.37109375
epoch 2 test acc 0.27734375
ep

In [74]:
train_digit3['target'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

# 추론

### digit_2 추론

In [61]:
df_test_digit2 = df_test
df_test_digit2

,text,target
0,경로당에서 사회적으로도움이필요한사람에게 복지서비스,
1,매장에서 일반인을 대상으로 여성의류판매,
2,자동화 기계 설비 공장 만듬,
3,사업장에서 소프트웨어개발 물류창고시스템,
4,철판.구리.전선 가공조립 수배전반,
...,...,...
995,사업장에서 일반인대상으로 건어물 판매,
996,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,
997,사업장에서 개인 소비자에게 월간 잡지 발행,
998,사업장에서 접객시설을 갖추고 참치회판매,


In [62]:
test2 = df_test_digit2
test2['target'] = ''
test2

,text,target
0,경로당에서 사회적으로도움이필요한사람에게 복지서비스,
1,매장에서 일반인을 대상으로 여성의류판매,
2,자동화 기계 설비 공장 만듬,
3,사업장에서 소프트웨어개발 물류창고시스템,
4,철판.구리.전선 가공조립 수배전반,
...,...,...
995,사업장에서 일반인대상으로 건어물 판매,
996,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,
997,사업장에서 개인 소비자에게 월간 잡지 발행,
998,사업장에서 접객시설을 갖추고 참치회판매,


In [64]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = test['target'].map(target_to_num_digit2)

tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220404_inference_digit2/model' + str(fold), num_labels=68)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict2.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label2 = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob2 = pd.DataFrame(output_prob)

  df_label2.to_csv('/content/drive/MyDrive/220404_inference_digit2/pred_label'+str(fold)+'.csv', index=False)
  df_prob2.to_csv('/content/drive/MyDrive/220404_inference_digit2/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [65]:
df_label2

,index,label
0,0,H 49
1,1,G 47
2,2,I 56
3,3,L 68
4,4,G 46
...,...,...
995,995,G 47
996,996,I 56
997,997,G 47
998,998,I 56


In [38]:
df_test__1 = df_test_1
df_test__1['target'] = df_test__1['digit_1'] + " " + df_test__1['digit_2'].astype(str)
df_test__1

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
9000,id_0045329,Q,87,872,경로당에서,사회적으로도움이필요한사람에게,복지서비스,경로당에서 사회적으로도움이필요한사람에게 복지서비스,Q 87
9001,id_0327368,G,47,474,매장에서,일반인을 대상으로,여성의류판매,매장에서 일반인을 대상으로 여성의류판매,G 47
9002,id_0328131,C,25,259,자동화 기계 설비,공장,만듬,자동화 기계 설비 공장 만듬,C 25
9003,id_0471406,J,58,582,사업장에서,소프트웨어개발,물류창고시스템,사업장에서 소프트웨어개발 물류창고시스템,J 58
9004,id_0683855,C,28,281,철판.구리.전선,가공조립,수배전반,철판.구리.전선 가공조립 수배전반,C 28
...,...,...,...,...,...,...,...,...,...
9995,id_0737769,G,47,472,사업장에서,일반인대상으로,건어물 판매,사업장에서 일반인대상으로 건어물 판매,G 47
9996,id_0997069,I,56,562,포장마차에서,접객시설을갖추고,생ㅇ선탕.소주,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,I 56
9997,id_0144206,J,58,581,사업장에서,개인 소비자에게,월간 잡지 발행,사업장에서 개인 소비자에게 월간 잡지 발행,J 58
9998,id_0464947,I,56,561,사업장에서,접객시설을 갖추고,참치회판매,사업장에서 접객시설을 갖추고 참치회판매,I 56


In [41]:
df_test__1 = df_test__1.reset_index(drop=True)
df_test__1 = df_test__1['target']

In [50]:
df_test__1

0      Q 87
1      G 47
2      C 25
3      J 58
4      C 28
       ... 
995    G 47
996    I 56
997    J 58
998    I 56
999    H 49
Name: target, Length: 1000, dtype: object

### digit3 추론

In [66]:
df_test_digit3 = df_test
df_test_digit3

,text,target
0,경로당에서 사회적으로도움이필요한사람에게 복지서비스,
1,매장에서 일반인을 대상으로 여성의류판매,
2,자동화 기계 설비 공장 만듬,
3,사업장에서 소프트웨어개발 물류창고시스템,
4,철판.구리.전선 가공조립 수배전반,
...,...,...
995,사업장에서 일반인대상으로 건어물 판매,
996,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,
997,사업장에서 개인 소비자에게 월간 잡지 발행,
998,사업장에서 접객시설을 갖추고 참치회판매,


In [67]:
test3 = df_test_digit3
test['target'] = ''
test3

,text,target
0,경로당에서 사회적으로도움이필요한사람에게 복지서비스,
1,매장에서 일반인을 대상으로 여성의류판매,
2,자동화 기계 설비 공장 만듬,
3,사업장에서 소프트웨어개발 물류창고시스템,
4,철판.구리.전선 가공조립 수배전반,
...,...,...
995,사업장에서 일반인대상으로 건어물 판매,
996,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,
997,사업장에서 개인 소비자에게 월간 잡지 발행,
998,사업장에서 접객시설을 갖추고 참치회판매,


In [79]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = test['target'].map(target_to_num_digit3)

tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 190
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220404_inference_digit3/model' + str(fold), num_labels=190)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict3.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label3 = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob3 = pd.DataFrame(output_prob)

  df_label3.to_csv('/content/drive/MyDrive/220404_inference_digit3/pred_label'+str(fold)+'.csv', index=False)
  df_prob3.to_csv('/content/drive/MyDrive/220404_inference_digit3/pred_prob'+str(fold)+'.csv', index=False)

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [119]:
df_test_2 = df_test_1
df_test_3 = df_test_1

In [120]:
df_test_2

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
9000,id_0045329,Q,87,872,경로당에서,사회적으로도움이필요한사람에게,복지서비스,경로당에서 사회적으로도움이필요한사람에게 복지서비스,Q 87872
9001,id_0327368,G,47,474,매장에서,일반인을 대상으로,여성의류판매,매장에서 일반인을 대상으로 여성의류판매,G 47474
9002,id_0328131,C,25,259,자동화 기계 설비,공장,만듬,자동화 기계 설비 공장 만듬,C 25259
9003,id_0471406,J,58,582,사업장에서,소프트웨어개발,물류창고시스템,사업장에서 소프트웨어개발 물류창고시스템,J 58582
9004,id_0683855,C,28,281,철판.구리.전선,가공조립,수배전반,철판.구리.전선 가공조립 수배전반,C 28281
...,...,...,...,...,...,...,...,...,...
9995,id_0737769,G,47,472,사업장에서,일반인대상으로,건어물 판매,사업장에서 일반인대상으로 건어물 판매,G 47472
9996,id_0997069,I,56,562,포장마차에서,접객시설을갖추고,생ㅇ선탕.소주,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,I 56562
9997,id_0144206,J,58,581,사업장에서,개인 소비자에게,월간 잡지 발행,사업장에서 개인 소비자에게 월간 잡지 발행,J 58581
9998,id_0464947,I,56,561,사업장에서,접객시설을 갖추고,참치회판매,사업장에서 접객시설을 갖추고 참치회판매,I 56561


In [121]:
df_test_2['text'] = df_test_2['text_obj'].astype(str) + " " + df_test_2['text_mthd'].astype(str) + " " + df_test_2['text_deal'].astype(str)
df_test_2['target'] = df_test_2['digit_1'].astype(str) + " " + df_test_2['digit_2'].astype(str)
df_test_2 = df_test_2[['text','target']]
df_test_2 = df_test_2.reset_index(drop=True)

In [122]:
df_test_2

,text,target
0,경로당에서 사회적으로도움이필요한사람에게 복지서비스,Q 87
1,매장에서 일반인을 대상으로 여성의류판매,G 47
2,자동화 기계 설비 공장 만듬,C 25
3,사업장에서 소프트웨어개발 물류창고시스템,J 58
4,철판.구리.전선 가공조립 수배전반,C 28
...,...,...
995,사업장에서 일반인대상으로 건어물 판매,G 47
996,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,I 56
997,사업장에서 개인 소비자에게 월간 잡지 발행,J 58
998,사업장에서 접객시설을 갖추고 참치회판매,I 56


In [139]:
display(df_test__2)
display(df_label2)

,text,label
0,경로당에서 사회적으로도움이필요한사람에게 복지서비스,Q 87
1,매장에서 일반인을 대상으로 여성의류판매,G 47
2,자동화 기계 설비 공장 만듬,C 25
3,사업장에서 소프트웨어개발 물류창고시스템,J 58
4,철판.구리.전선 가공조립 수배전반,C 28
...,...,...
995,사업장에서 일반인대상으로 건어물 판매,G 47
996,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,I 56
997,사업장에서 개인 소비자에게 월간 잡지 발행,J 58
998,사업장에서 접객시설을 갖추고 참치회판매,I 56


,index,label
0,0,H 49
1,1,G 47
2,2,I 56
3,3,L 68
4,4,G 46
...,...,...
995,995,G 47
996,996,I 56
997,997,G 47
998,998,I 56


In [127]:
tdf_test__2 = df_test_2.rename(columns={'target':'label'})
df_test__2

,text,label
0,경로당에서 사회적으로도움이필요한사람에게 복지서비스,Q 87
1,매장에서 일반인을 대상으로 여성의류판매,G 47
2,자동화 기계 설비 공장 만듬,C 25
3,사업장에서 소프트웨어개발 물류창고시스템,J 58
4,철판.구리.전선 가공조립 수배전반,C 28
...,...,...
995,사업장에서 일반인대상으로 건어물 판매,G 47
996,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,I 56
997,사업장에서 개인 소비자에게 월간 잡지 발행,J 58
998,사업장에서 접객시설을 갖추고 참치회판매,I 56


In [148]:
df_test__2['label_2'] = df_label2['label']
len(df_test__2[df_test__2['label']==df_test__2['label_2']])/len(df_test__2)

0.462

In [140]:
sum = 0

for i in range(1000):
  if df_test__2['label'][i] == df_label2['label'][i] : 
    sum += 1
print(sum)

462


In [124]:
df_test_3['text'] = df_test_3['text_obj'].astype(str) + " " + df_test_3['text_mthd'].astype(str) + " " + df_test_3['text_deal'].astype(str)
df_test_3['target'] = df_test_3['digit_1'].astype(str) + " " + df_test_3['digit_2'].astype(str)+ " "+ df_test_3['digit_3'].astype(str)
df_test_3 = df_test_3[['text','target']]
df_test_3 = df_test_3.reset_index(drop=True)

In [125]:
display(df_test_3)
display(df_label3)

,text,target
0,경로당에서 사회적으로도움이필요한사람에게 복지서비스,Q 87 872
1,매장에서 일반인을 대상으로 여성의류판매,G 47 474
2,자동화 기계 설비 공장 만듬,C 25 259
3,사업장에서 소프트웨어개발 물류창고시스템,J 58 582
4,철판.구리.전선 가공조립 수배전반,C 28 281
...,...,...
995,사업장에서 일반인대상으로 건어물 판매,G 47 472
996,포장마차에서 접객시설을갖추고 생ㅇ선탕.소주,I 56 562
997,사업장에서 개인 소비자에게 월간 잡지 발행,J 58 581
998,사업장에서 접객시설을 갖추고 참치회판매,I 56 561


,index,label
0,0,G 47 475
1,1,G 47 475
2,2,I 56 562
3,3,G 47 475
4,4,I 56 562
...,...,...
995,995,G 47 475
996,996,I 56 562
997,997,G 47 475
998,998,I 56 562


In [141]:
df_test_3['label'] = df_label3['label']

In [143]:
len(df_test_3[df_test_3['target']==df_test_3['label']])/len(df_test_3)

0.054

In [ ]:
sum = 0

for i in range(1000):
  if df_test__3['label'][i] == df_label3['label'][i] : 
    sum += 1
print(sum)

In [126]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label4.csv')

In [ ]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']
df = test

In [ ]:
df.head(3)

In [ ]:
1-((len(df)-len(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])]))/len(df))

In [ ]:
hard = df[df.merge(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])], how = 'left', left_index=True, right_index=True)['text_y'].isnull()]
hard

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob4.csv')

In [ ]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test = pd.concat([test, pred], axis=1)
test.head(3)

In [ ]:
def num_to_label(label):
    label_dict = label_dict = {v: k for k, v in my_dict.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

test['soft'] = pd.DataFrame(answer)[1]
test.head(3)

In [ ]:
soft = test[['label0', 'label1', 'label2', 'label3', 'label4', 'soft']]
soft

In [ ]:
def label_count(df):
  num_count = {}
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    try: num_count[df[col]] += 1
    except: num_count[df[col]] = 1
  return(num_count)

temp = soft.apply(label_count, axis=1)
soft['temp'] = temp

def label_count(temp:dict):
  keys = []
  temptemp = 0
  for k, v in temp.items():
    if v >= 3:
      temptemp = k
  keys.append(temptemp)
  return keys[0]
  
soft['hard'] = soft['temp'].map(label_count)

In [ ]:
soft[soft['soft'] != soft['hard']]

In [ ]:
df = soft[['label0',	'label1',	'label2',	'label3',	'label4',	'soft',	'hard']]
df

In [ ]:
submission

In [ ]:
print(my_variable)

In [ ]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = df['soft'].map(split_label)

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220331_inference/submission.csv', index=False)